# Building a song recommender to users using collaborative filtering and KNN


#import library ต่างๆ และเชื่อมฐานข้อมูล

In [1]:
import mysql.connector
import pandas as pd

db_connection = mysql.connector.connect(
    host="l3ackstab.thddns.net",
    port="7676",
    user="kitti",
    passwd="961016",
    database="music_db"
)


Query ดาต้าด้วยคำสั่ง SQL

In [2]:
query = """
SELECT pc.song_id, s.title, pc.user_id, pc.click_count
FROM play_counts pc
JOIN songs s ON pc.song_id = s.song_id
WHERE pc.user_id != 0
"""

cursor = db_connection.cursor()
cursor.execute(query)
rows = cursor.fetchall()



เก็บข้อมูลจาก MySQL เป็น DataFrame

In [3]:
# แปลงเป็น DataFrame
song_data = pd.DataFrame(rows, columns=['song_id', 'title', 'user_id', 'click_count'])

# Explore data

แสดงข้อมูล

In [4]:
song_data.head()

,song_id,title,user_id,click_count
0,39,ถ้าเธอรักฉันจริง,16,2
1,39,ถ้าเธอรักฉันจริง,50,1
2,39,ถ้าเธอรักฉันจริง,66,1
3,39,ถ้าเธอรักฉันจริง,68,1
4,40,ฝนตกไหม,16,1


แสดงเพลงที่มีการคลิกมากสุดในดาต้าเซ็ท

In [5]:
song_data['title'].value_counts()

title
ไม่อยากเหงาแล้ว               21
17                            16
ช่วงเวลา                      16
ลองเลิกกันดีไหม               15
ซินเดอเรลล่า                  13
                              ..
 ระบายเฉยๆ                     1
ชีวิตเป็นของเรา                1
เทพลีลา                        1
แกล้ง                          1
Can't Take My Eyes Off You     1
Name: count, Length: 105, dtype: int64

In [6]:
len(song_data)

678

นับ user ทั้งหมดในดาต้าเซ็ท

In [7]:
users = song_data['user_id'].unique()

In [8]:
len(users)

70


# Create a song recommender

# Using Collaborative Filtering

สร้างเมทริกซ์ที่มีจำนวนผู้ใช้ * จำนวนของเนื้อเพลง
แต่ละรายการในอาร์เรย์แสดงถึงจำนวนครั้งที่เพลงที่ผู้ใช้รายหนึ่งคลิก

In [9]:
song=song_data.drop_duplicates(['user_id','title'])
song=song.reset_index(drop=True)

ใช้เพียง 50,000 รายการใน song_data (jupyter notebook มีหน่วยความจำจำกัด)

In [10]:
song=song.iloc[0:50000,]
song.shape

(678, 4)

In [11]:
song.head()

,song_id,title,user_id,click_count
0,39,ถ้าเธอรักฉันจริง,16,2
1,39,ถ้าเธอรักฉันจริง,50,1
2,39,ถ้าเธอรักฉันจริง,66,1
3,39,ถ้าเธอรักฉันจริง,68,1
4,40,ฝนตกไหม,16,1


ใช้กรองตาราง

In [12]:
song_pivot=song.pivot(index='user_id',columns='song_id',values='click_count')

In [13]:
song_pivot.shape

(70, 106)

In [14]:
song_pivot=song_pivot.fillna(0)

เลือก user ที่มีการคลิก song_id = 57 มากกว่าหรือเท่ากับ 1 ครั้ง (ในตัวอย่างจะใช้ ==)

In [15]:
song_pivot[song_pivot[57] == 1]


song_id,39,40,41,42,43,44,45,46,49,50,...,145,146,147,148,149,150,151,152,153,155
user_id,,,,,,,,,,,,,,,,,,,,,
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,2.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Train a Knn model โดยใช้ pivot_table

In [16]:
from sklearn.neighbors import NearestNeighbors
knn=NearestNeighbors(n_neighbors=5,metric='cosine')
Model=knn.fit(song_pivot)

In [17]:
song_pivot[song_pivot.index == 1]

song_id,39,40,41,42,43,44,45,46,49,50,...,145,146,147,148,149,150,151,152,153,155
user_id,,,,,,,,,,,,,,,,,,,,,


# Predict neighbors for สำหรับเจาะจง user

In [18]:
user = song_pivot.iloc[1]

In [19]:
distances, indices = Model.kneighbors([user])
distances, indices

(array([[0.        , 0.5669873 , 0.59175171, 0.61075053, 0.61270167]]),
 array([[ 1,  8, 23,  2, 20]], dtype=int64))

indices มีเพียงค่าดัชนีของ users จากตาราง user. จะไม่โชว์ user_id.
จะต้องหา user_id ของ neighbors

In [20]:
neighbors=[]
for item in indices[0][1:]:
    neighbors.append(song[song.index == item].user_id.values[0])
neighbors

[66, 39, 66, 14]

สร้าง dataframe พร้อมรายละเอียดของ neighbors เท่านั้น

In [21]:
# ตั้งค่ารายการเปล่าสำหรับเก็บ DataFrame ชั่วคราว
temp_dfs = []

# วนซ้ำผ่านรายการของ neighbors
for item in neighbors:
    # กรองข้อมูลจาก DataFrame `song` และเก็บในรายการ temp_dfs
    filtered_df = song[song.user_id == item]
    temp_dfs.append(filtered_df)

# ใช้ pd.concat ในการรวมรายการของ DataFrame เหล่านั้นเข้าด้วยกัน
neighbor_songs = pd.concat(temp_dfs, ignore_index=True)

# ตรวจสอบข้อมูลใน neighbor_songs
neighbor_songs


,song_id,title,user_id,click_count
0,39,ถ้าเธอรักฉันจริง,66,1
1,40,ฝนตกไหม,66,1
2,41,ข้างกัน (City),66,2
3,43,เปิดตัวเขา,66,1
4,46,น้อง,66,1
5,44,ฝันถึงแฟนเก่า,39,1
6,71,หยดน้ำตา,39,1
7,83,The Weekend,39,1
8,98,เป็นทุกอย่าง,39,1
9,118,ยาพิษ,39,1


ในแต่ละ song ใน neighbor_songs หาจำนวนที่คลิกทั้งหมดจาก neighbors ทั้งหมด

In [22]:
# รวมข้อมูลจาก temp_dfs และสร้าง neighbor_songs
neighbor_songs = pd.concat(temp_dfs, ignore_index=True)

# สร้าง DataFrame ใหม่โดยใช้คอลัมน์ click_count และ title
neighbor_songs = pd.DataFrame({
    'Count': neighbor_songs['click_count'],
    'Song': neighbor_songs['title']  # ใช้คอลัมน์ title จาก DataFrame ที่รวมมา
})
neighbor_songs

,Count,Song
0,1,ถ้าเธอรักฉันจริง
1,1,ฝนตกไหม
2,2,ข้างกัน (City)
3,1,เปิดตัวเขา
4,1,น้อง
5,1,ฝันถึงแฟนเก่า
6,1,หยดน้ำตา
7,1,The Weekend
8,1,เป็นทุกอย่าง
9,1,ยาพิษ


In [23]:
# เรียงลำดับข้อมูลตาม Count และสร้างใหม่ถ้าต้องการ
neighbor_songs = neighbor_songs.sort_values('Count', ascending=False)
neighbor_songs

,Count,Song
43,4,ขาหมู
23,3,ไม่อยากเหงาแล้ว
18,3,ดาวหางฮัลเลย์
41,3,ฝากที
24,2,อยากเริ่มต้นใหม่กับคนเดิม
13,2,ข้างกัน (City)
25,2,ลบไม่ได้ช่วยให้ลืม
22,2,ลองเลิกกันดีไหม
21,2,17
35,2,ข้อยกเว้น


นำเรียงลำดับข้อมูล

In [24]:
neighbor_songs=neighbor_songs.reset_index(drop=True)
neighbor_songs

,Count,Song
0,4,ขาหมู
1,3,ไม่อยากเหงาแล้ว
2,3,ดาวหางฮัลเลย์
3,3,ฝากที
4,2,อยากเริ่มต้นใหม่กับคนเดิม
5,2,ข้างกัน (City)
6,2,ลบไม่ได้ช่วยให้ลืม
7,2,ลองเลิกกันดีไหม
8,2,17
9,2,ข้อยกเว้น


ทำการแนะนำเพลง โดยจะแนะนำเพลงจากความนิยมทั้งหมด ไม่เฉพาะเจาะจง user

In [25]:
Recommended_Song=neighbor_songs['Song'][0:10]
print(Recommended_Song)

0                        ขาหมู
1              ไม่อยากเหงาแล้ว
2                ดาวหางฮัลเลย์
3                        ฝากที
4    อยากเริ่มต้นใหม่กับคนเดิม
5               ข้างกัน (City)
6           ลบไม่ได้ช่วยให้ลืม
7              ลองเลิกกันดีไหม
8                           17
9                    ข้อยกเว้น
Name: Song, dtype: object


# The Top 10 song reccomendations for the user!!!

การแนะนำแบบเฉพาะเจาะจง user

In [26]:
# เลือก user_id สำหรับการนำเสนอ
user_id_specific = 18  # ตัวอย่าง user_id ที่คุณต้องการแสดงการแนะนำ

In [27]:
# หา user_vector สำหรับ user_id ที่เลือก
user_vector = song_pivot.loc[user_id_specific].values.reshape(1, -1)
user_vector

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 2., 0., 2., 3., 4., 2., 2., 3., 1., 0., 2., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [28]:
# คำนวณหาเพื่อนบ้านที่ใกล้ที่สุด
distances, indices = Model.kneighbors(user_vector)

In [29]:

# แสดงเพลงที่แนะนำโดยอิงจาก indices ของเพลง
recommended_song_ids = song_pivot.columns[indices.flatten()[1:]]
recommended_songs = song_data[song_data['song_id'].isin(recommended_song_ids)]['title'].drop_duplicates()

print(f"เพลงที่แนะนำสำหรับ user_id {user_id_specific}:")
for song in recommended_songs:
    print(song)


เพลงที่แนะนำสำหรับ user_id 18:
17
พักก่อน
สาธุ
โกหก
